# Neural Network Assignment
## Report
*by Lethabo Neo* 

# Imports, Installations and Downloads

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset,random_split
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

I added random_spilt for random split of train,validation and test data.

# Data Processing
- Reading files from directory
- Converting the images into vectors
- Displaying the images
- Subsetting the data
- Assigning the kind of data to labels

1. Read files from directory

In [11]:
DATA_DIR = "/Users/lethaboneo/Desktop/Computer Science/CSC3022F/Assignments/Machine Learning/ML Assignment 1/"
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
train = datasets.FashionMNIST(DATA_DIR, train=True, download=False, transform=transform)
test = datasets.FashionMNIST(DATA_DIR, train=False, download=False,transform=transform)

*Visualisation purposes*
*print the datasets*

In [ ]:
print("Train dataset:", train)
print("Test dataset:", test)

In [ ]:
train_size= int(0.9*len(train))
val_size= len(train)- train_size
trainData, valData = random_split(train,[train_size,val_size])
print("Training Dataset:", trainData)
print("Validation Dataset:", valData)
print("Test Dataset:", test)

Split data into Training/Validation/Test sets


In [14]:
# Create data loaders
batch_size = 128

train_loader = torch.utils.data.DataLoader(dataset=trainData, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=valData, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)


Split the data in batches for more efficient and fater training. Process 128 images at a time

# Neural Network implementation


In [ ]:

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size1 , num_classes):
        super(NeuralNetwork,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.bn= nn.BatchNorm1d(hidden_size)
        self.dropout=nn.Dropout(0.3)
        self.relu =nn.ReLU()

        self.fc2 = nn.Linear(hidden_size,hidden_size1)
        self.bn1= nn.BatchNorm1d(hidden_size1)
        self.dropout1=nn.Dropout(0.5)
        self.relu1 =nn.ReLU()
        self.fc3 = nn.Linear(hidden_size1, num_classes)

    def forward(self,x):
        #1 hidden layer
        out =self.fc1(x)
        out =self.bn(out)
        out= self.relu(out)
        out = self.dropout(out)
        #2 hidden layer
        out = self.fc2(out)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.dropout(out)

        out = self.fc3(out)
        return out
        

# Details
My final model is 1 hidden layer Neural Network.



# Train model
*Initilaise Hyperparamters*


In [22]:
learn_rate = 0.001
num_epochs = 30
input_size = 28*28
hidden_size = 256
hidden_size1 = 128

num_classes = 10


train_loss = []
val_loss = []
val_acc = []
best_val_acc = 0
model = NeuralNetwork(input_size, hidden_size,num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learn_rate,weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=2,factor=0.5)
print(model)

TypeError: NeuralNetwork.__init__() missing 1 required positional argument: 'num_classes'

# Validation phase

In [ ]:

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.reshape(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Calculate epoch training loss
    epoch_loss = running_loss / len(train_loader)
    train_loss.append(epoch_loss)
    
    # Validation phase
    model.eval()
    val_loss_epoch = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.reshape(-1, input_size)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss_epoch += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Calculate validation metrics
    val_loss_epoch /= len(val_loader)
    val_loss.append(val_loss_epoch)
    val_accuracy = 100 * correct / total
    val_acc.append(val_accuracy)
    
    # Update learning rate
    scheduler.step(val_accuracy)
    
    # Save best model
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        torch.save(model.state_dict(), 'best_model_nn.pth')
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {epoch_loss:.4f}, '
          f'Val Loss: {val_loss_epoch:.4f}, '
          f'Val Accuracy: {val_accuracy:.2f}% '
          f'LR: {optimizer.param_groups[0]["lr"]:.6f}')

# Evaluate performance

In [19]:
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.reshape(-1, 28*28)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

print("\nFinal Evaluation:")
val_accuracy = evaluate_model(model, val_loader)
test_accuracy = evaluate_model(model, test_loader)
print(f'Validation Accuracy: {val_accuracy:.2f}%')
print(f'Test Accuracy: {test_accuracy:.2f}%')


Final Evaluation:
Validation Accuracy: 90.12%
Test Accuracy: 89.88%


# Graphs

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

plt.subplot(1, 2, 2)
plt.plot(test, label='Tr Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import itertools
def evaluate_model_comprehensive(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.reshape(-1, 28*28)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return all_labels, all_preds

# Get predictions for validation set
val_labels, val_preds = evaluate_model_comprehensive(model, val_loader)

# Get predictions for test set
test_labels, test_preds = evaluate_model_comprehensive(model, test_loader)

# Classification Report
print("\nValidation Set Classification Report:")
print(classification_report(val_labels, val_preds, target_names=[
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]))

print("\nTest Set Classification Report:")
print(classification_report(test_labels, test_preds, target_names=[
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]))

# Confusion Matrix Visualization
def plot_confusion_matrix(labels, preds, title):
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', 
                xticklabels=[
                    'T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
                    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot'
                ],
                yticklabels=[
                    'T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
                    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot'
                ])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

print("\nValidation Set Confusion Matrix:")
plot_confusion_matrix(val_labels, val_preds, "Validation Set Confusion Matrix")

print("\nTest Set Confusion Matrix:")
plot_confusion_matrix(test_labels, test_preds, "Test Set Confusion Matrix")
